<a href="https://colab.research.google.com/github/stefdworschak/ai-business-app/blob/vectorstore-and-agent/solve_business_problems_with_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [11]:
# Install the OpenAI Python package and LangChain components.
%pip install openai httpx==0.27.2 langchain-pinecone langchain-text-splitters

In [12]:
# Import secrets from the environment
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
OPENAI_BASE = userdata.get('OPENAI_BASE')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [13]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [14]:
# Code example of generating output document

# Creates a new shared Google Worksheet every invocation with the proper structure
# Note: This code should be executed from the google colab once you are ready, it will not work locally
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# Authenticate user
auth.authenticate_user()
creds, _ = default()

gc = gspread.authorize(creds)

# IMPORTANT: You need to authenticate the user to be able to create new worksheet
# Insert the authentication snippet from the official documentation to create a google client:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=qzi9VsEqzI-o

# This code goes after creating google client
#output_document = gc.create('Solving Business Problems with AI - Output')
output_document = gc.open_by_key('1N7h0_zDvY_WtEyi50PpxWt56JRSfUBHl6Ih-ib8_D3g')

# Create 'email-classification' sheet
#email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet = output_document.worksheet(title="email-classification")
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')

# Example of writing the data into the sheet
# Assuming you have your classification in the email_classification_df DataFrame
# set_with_dataframe(email_classification_sheet, email_classification_df)
# Or directly update cells: https://docs.gspread.org/en/latest/user-guide.html#updating-cells

# Create 'order-status' sheet
#order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet = output_document.worksheet(title="order-status")
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')

# Create 'order-response' sheet
#order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet = output_document.worksheet(title="order-response")
order_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Create 'inquiry-response' sheet
#inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet = output_document.worksheet(title="inquiry-response")
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')

# This is the solution output link, paste it into the submission form
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")

Shareable link: https://docs.google.com/spreadsheets/d/1N7h0_zDvY_WtEyi50PpxWt56JRSfUBHl6Ih-ib8_D3g


# Setup 1: Create VectorstoreFactory class for managing the Pinecone Vectorstore

In [15]:
from uuid import uuid4

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pinecone import Pinecone, ServerlessSpec


class VectorstoreFactory:
    def __init__(self, pinecone):
        self.pinecone = pinecone
        self.index = None
        self.embeddings = None
        self.vector_store = None

    def get_or_create_index(self, index_name, dimension=768):
        if not self.pinecone.has_index(index_name):
            self.pinecone.create_index(
                name=index_name,
                dimension=1536,
                metric="cosine",
                spec=ServerlessSpec(cloud="aws", region="us-east-1"),
            )

        self.index = self.pinecone.Index(index_name)
        return self.index

    def get_embeddings(self):
        self.embeddings = OpenAIEmbeddings(
            model='text-embedding-3-small',
            openai_api_key=OPENAI_API_KEY,
            openai_api_base=OPENAI_BASE,
        )
        return self.embeddings

    def get_vectorstore(self, index_name):
        self.vector_store =  PineconeVectorStore(
            index=self.get_or_create_index(index_name),
            embedding=self.get_embeddings())
        return self.vector_store

    def create_document(self, page_content, metadata={}):
        return Document(
            page_content=page_content,
            metadata=metadata
        )

    def chunk_text(self, text, chunk_size=500, chunk_overlap=50):
        """Chunk text into smaller pieces for better embedding performance."""
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
        return text_splitter.split_text(text)

    def create_chunked_documents(self, text, metadata={}, chunk_size=500, chunk_overlap=50):
        """Create multiple documents from chunked text."""
        chunks = self.chunk_text(text, chunk_size, chunk_overlap)
        documents = []

        for i, chunk in enumerate(chunks):
            chunk_metadata = metadata.copy()
            chunk_metadata['chunk_id'] = i
            chunk_metadata['total_chunks'] = len(chunks)
            documents.append(self.create_document(chunk, chunk_metadata))

        return documents

    def add_documents(self, documents, index_name):
        uuids = [str(uuid4()) for _ in range(len(documents))]
        self.vector_store.add_documents(documents=documents, ids=uuids)

# Setup 2: Create BusinessAgent class for email classification and order analysis using Retrieval Augmented Generation (RAG)

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Literal, List


# Define structured output schemas using Pydantic
class EmailClassification(BaseModel):
    """Schema for email classification results"""
    category: Literal["product inquiry", "order request"]
    confidence: float  # Between 0.0 and 1.0
    reasoning: str     # Brief explanation of the classification


class BusinessAgent:
    def __init__(self, client, vector_store):
        self.client = client

    def classify_email_structured(self, email_subject: str, email_text: str, model: str = "gpt-4o") -> EmailClassification:
        """
        Classify an email using structured output to ensure consistent format
        """
        system_prompt = """
        You are an AI assistant helping classify customer emails for a fashion store.

        Your task is to analyze emails and classify them as either:
        - "product inquiry": Customer asking questions about products, availability, features, etc.
        - "order request": Customer wanting to purchase, order, or buy specific items

        Provide a confidence score between 0.0 and 1.0 and explain your reasoning briefly.
        """

        response = self.client.beta.chat.completions.parse(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Please classify this email:\n\nSubject: {email_subject}\n\nBody: {email_text}"}
            ],
            response_format=EmailClassification
        )

        return response.choices[0].message.parsed

    def classify_emails(self, email_text, model="gpt-4o"):
        """Legacy method - kept for backwards compatibility"""
        result = self.classify_email_structured(email_text, model)
        return result.category

In [21]:
from langchain.tools import tool

from langchain.tools import StructuredTool
from pydantic import BaseModel

# Step 1: Define your class
class StockManager:
    def __init__(self, stock_count: dict):
        self.stock_count = stock_count

    @tool
    def get_stock_count(self, product_id: str) -> int:
        """ Get the number of items left in stock for a product based on its product_id """
        return self.stock_count[input.product_id]

# Task 1. Classify emails

In [ ]:
# Recreate vectorstore factory to clear any cached parameters
# This ensures we use the updated vectorstore.py without problematic parameters
from pinecone import Pinecone

index_name = 'products'
# Recreate the vectorstore factory with fresh instance
pinecone = Pinecone(api_key=PINECONE_API_KEY)
vectorstore_factory = VectorstoreFactory(pinecone=pinecone)
vectorstore = vectorstore_factory.get_vectorstore(index_name=index_name)

print("Vectorstore factory recreated successfully!")

Vectorstore factory recreated successfully!


In [ ]:
client = OpenAI(
    base_url=OPENAI_BASE,
    api_key=OPENAI_API_KEY,
)

agent = BusinessAgent(client, vector_store=vectorstore)

In [ ]:
# Classify emails using structured output
for idx, row in emails_df.iterrows():
    # Get structured classification result
    classification = agent.classify_email_structured(row['subject'], row['message'])

    # Store the category in the dataframe
    emails_df.loc[idx, 'category'] = classification.category

# Write orders to sheet
classified_emails_output_df = emails_df[['email_id', 'category']]
classified_emails_output = classified_emails_output_df.values.tolist()
email_classification_sheet.update(classified_emails_output, f'A2:B{len(classified_emails_output)+1}')

,email_id,subject,message,category
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT...",order request
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V...",order request
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In...",order request
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry


# Setup 3: Setup Stock Manager and OrderAgent

In [75]:
#Process orders

# - For each order request, verify product availability in stock.
# - If the order can be fulfilled, create a new order line with the status “created”.
# - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
# - Update stock levels after processing each order.
# - Record each product request from the email.
# - Output: Populate the order-status sheet with columns: email ID, product ID, quantity, status ("created", "out of stock").
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor


class OrderItem(BaseModel):
    """Schema for individual order items"""
    product_mentioned: str    # Product name/description mentioned by customer
    product_id: str          # Matched product ID from catalog
    order_quantity: int            # Quantity requested


class OrderAnalysis(BaseModel):
    """Schema for analyzing order requests"""
    order_items: List[OrderItem]  # List of ordered items
    customer_intent: str          # What the customer wants to do
    urgency_level: Literal["low", "medium", "high"]
    status: Literal["created", "out of stock"]


class OrderAgent:
    def __init__(self, llm, vector_store, stock_manager):
        self.llm = llm
        self.vector_store = vector_store
        self.stock_manager = stock_manager
        self.tools = self.create_tools()
        self.agent = self.setup_agent()

    def create_tools(self):
        return [
            self.stock_manager.get_stock_count,
        ]

    @staticmethod
    def get_system_prompt():
        return """
            You are analyzing order request emails for a fashion store.
            Extract the following information for each product mentioned:
            - Create an OrderItem for each product the customer wants to order
            - For each OrderItem, include:
              * product_mentioned: The exact product name/description the customer used
              * product_id: The matching product ID from the catalog below
              * order quantity: How many items. If no order quantity specified, but terms such as `all the remaining stock`,
                `whatever is left` or `all you got` are used, try to derive the quantity from the existing stock count.

            Also determine:
            - status: Based on the number of items left in stock and the order quantity whether there is enough stock to fulfil the customer order
              and mark the appropriate status (created, out of stock)
            - customer_intent: What the customer wants to do overall
            - urgency_level: Based on language used (low/medium/high)

            To determine stock levels you have access to the following tools:
            - get_stock_count(product_id: str) -> int: Use this tool to get the number of items left in stock for a product based on its product_id

            Use reason to think about what you need to do step by step. Follow the ReAct methodology to analyze the order.

            Make sure to match products based on name, category, season, and description.
            If multiple products are mentioned, create separate OrderItem entries for each.

            Return your analysis in JSON format between three backticks (`) in the following format and nothign else:
            ```
            {
              "order_items": [
                {
                      "product_mentioned": str,
                      "product_id": str,
                      "order_quantity": int
                    }
                  ]
                  "customer_intent": str,
                  "urgency_level": str,
                  "status": "str
              ]
            }
            ```

            Here are some examples of emails you might encounter:

            Example 1:
              Email
                Hi there, I would like to order all of your remaining red velcrow scrafs
              Product Catalog Provided:
                Product ID: 122 | Name: Baseball cap | Category: hats | Seasons: spring, summer | Description: Black baseball cap for summer occasions
                Product ID: 123 | Name: Red scarf | Category: scarfs | Seasons: autumn | Description: Soft red velcrow scarf
              Tools available:
                get_stock_count(product_id: str) -> int: Use this tool to get the number of items left in stock for a product based on its product_id
              Reason:
                The customer mentions red velcrow scarfs, there is a product called `Red scarf` with Product ID `123` in the Product Catalog. That should be the right item.
                The customer does not mention a specific quantity, but wants the remaining red velcrow scarfs. I can use the tool `get_stock_count` to retrieve the order_quantity
              Tool Use
                Calling: get_stock_count('123')
                Returns: 3
              Observe:
                There are 3 `Red scarf` items left. Since there are still 3 `Red scarf` items left, the status should be set to `created`
              Final Answer:
                ```{
                  "order_items": [
                    {
                      "product_mentioned": "Red scarf",
                      "product_id": "123",
                      "order_quantity": 3
                    }
                  ]
                  "customer_intent": "The customer wants to order 3 red velcrow scarfs (Product ID: 123)",
                  "urgency_level": "low",
                  "status": "created"
                }```

            Example 2:
              Email
                Hi there, I would like to five baseball hats (122)
              Product Catalog Provided:
                Product ID: 122 | Name: Baseball cap | Category: hats | Seasons: spring, summer | Description: Black baseball cap for summer occasions
                Product ID: 123 | Name: Red scarf | Category: scarfs | Seasons: autumn | Description: Soft red velcrow scarf
              Tools available:
                get_stock_count(product_id: str) -> int: Use this tool to get the number of items left in stock for a product based on its product_id
              Reason:
                The customer wants to order 5 baseball hats with the product ID 122. The product name for product ID is `Baseball cap`
                The customer says they want to order 5 `Baseball cap` items. I can use the tool `get_stock_count` to check how much stock is left for this item
              Tool Use
                Calling: get_stock_count('122')
                Returns: 0
              Observe:
                There are no `Baseball cap` items left. Since there are no `Baseball cap` items left, the status should be set to `out of stock`
              Final Answer:
                ```{
                  "order_items": [
                    {
                      "product_mentioned": "Baseball cap",
                      "product_id": "122",
                      "order_quantity": 5
                    }
                  ]
                  "customer_intent": "The customer wants to order 3 red velcrow scarfs (Product ID: 123)",
                  "urgency_level": "medium",
                  "status": "out of stock"
                }```
        """

    def setup_agent(self):
        prompt = hub.pull("hwchase17/react")
        prompt.template += self.get_system_prompt()
        agent = create_react_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=prompt,
        )

        return AgentExecutor(agent=agent, tools=self.tools, verbose=True)

    def analyze_order_request(self, email_subject: str, email_text: str, model: str = "gpt-4o") -> OrderAnalysis:
        """
        Analyze an order request email to extract structured product information
        """
        prompt = """
        Analyze the following order email and extract Order Items for each product the customer wants to order:

        {email_content}

        Use the product catalog below to find the most relevant product ID for each item:

        {retrieved_context}
        """

        retriever = self.vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5}
        )

        email_content = f"Subject: {email_subject}\n\nBody: {email_text}"
        retrieved_docs = retriever.invoke(email_content)

        # Format the retrieved context to include metadata fields
        context_parts = []
        for doc in retrieved_docs:
            metadata = doc.metadata
            context_part = f"""
Product ID: {metadata.get('product_id', 'N/A')}
Name: {metadata.get('name', 'N/A')}
Category: {metadata.get('category', 'N/A')}
Seasons: {metadata.get('seasons', 'N/A')}
Description: {doc.page_content}

---"""
            context_parts.append(context_part)

        retrieved_context = "\n".join(context_parts)
        prompt = self.get_system_prompt() + prompt.format(email_content=email_content, retrieved_context=retrieved_context)
        response = self.agent.invoke({'input': prompt})
        return response


# Task 2. Process order requests

In [26]:
# TODO: Remove for the final version
email_classification = email_classification_sheet.get_values()
columns = email_classification.pop(0)
email_categories = pd.DataFrame(data=email_classification, columns=columns)
email_categories_orders_only = email_categories[email_categories['category']=='order request']
order_emails = emails_df[emails_df['email_id'].isin(email_categories_orders_only['email ID'])]
order_emails.head()

,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In..."
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl..."
7,E008,Ordering a Versatile Scarf-like item,"Hello, I'd want to order one of your Versatile..."


In [76]:
# Create order_agent
from langchain_openai import ChatOpenAI

stock_count = {row["product_id"]: row["stock"] for _, row in products_df.iterrows()}
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4",
    temperature=0,
    model_kwargs={
      #'system_message': OrderAgent.get_system_prompt(),
      'openai_base_url': OPENAI_BASE,
    },
)

stock_manager = StockManager(stock_count)

order_agent = OrderAgent(llm, vectorstore_factory.vector_store, stock_manager)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [78]:
order_request1 = order_agent.analyze_order_request(order_emails.loc[0]['subject'], order_emails.loc[0]['message'])



> Entering new AgentExecutor chain...


KeyError: 'Input to PromptTemplate is missing variables {\'\\n              "order_items"\', \'\\n                  "order_items"\'}.  Expected: [\'\\n                  "order_items"\', \'\\n              "order_items"\', \'agent_scratchpad\', \'input\'] Received: [\'input\', \'intermediate_steps\', \'agent_scratchpad\']\nNote: if you intended {\n              "order_items"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n              "order_items"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [ ]:
# Creating documents from products DataFrame and store them in the vector store
documents = []
for _, row in products_df.iterrows():
    product_metadata = {
        'product_id': row['product_id'],
        'name': row['name'],
        'category': row['category'],
        'price': row['price'],
        'seasons': row['seasons'],
    }
    documents.append(vectorstore_factory.create_document(page_content=row['description'], metadata=product_metadata))

vectorstore_factory.add_documents(documents, index_name=index_name)

# Task 3. Handle product inquiry